In [1]:
import time 
t_script = time.time()

### Load libraries, functions, palette, theme

In [2]:
%run __libraries.ipynb

In [3]:
%run __functions.ipynb

## Dataset Generation

### Generate Data

In [4]:
start_date = '2022-05-02'

In [5]:
customers, terminals, data = generate_dataset(
    start_date=start_date,
    random_state=seed,
    n_customers=5000, 
    n_terminals=10000, 
    nb_days=217, 
    r=5
)

In [6]:
data = add_frauds(customers, terminals, data)

In [7]:
data.head()

,transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
0,0,2022-05-02 00:00:31,1829,2355,22.35,31,0,0,0
1,1,2022-05-02 00:01:59,157,7040,9.53,119,0,0,0
2,2,2022-05-02 00:02:29,834,2170,82.26,149,0,0,0
3,3,2022-05-02 00:04:47,2845,8301,25.29,287,0,0,0
4,4,2022-05-02 00:07:04,1565,9269,31.69,424,0,0,0


In [8]:
data.shape

(2107260, 9)

In [9]:
data['tx_fraud'].sum()

17752

In [10]:
data['tx_fraud'].sum() / len(data['tx_fraud']) * 100

0.8424209637159153

In [11]:
DIR_OUTPUT = './data/'

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')

for day in range(data['tx_time_days'].max()+1):
    
    transactions_day = data[data['tx_time_days']==day].sort_values('tx_time_seconds')
    
    date = start_date + dt.timedelta(days=day)
    filename_output = date.strftime('%Y-%m-%d')+'.pkl'
    
    # Protocol=4 required for Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)

### Transform Data

#### Load Data

In [12]:
start_date = '2022-05-02'
end_date = '2022-11-30'

In [13]:
data = read_from_files(
    DIR_INPUT = './data/',
    BEGIN_DATE = start_date,
    END_DATE = end_date,
    sort='transaction_id'
)

In [14]:
data['tx_during_weekend'] = data['tx_datetime'].apply(is_weekend)
data['tx_during_night'] = data['tx_datetime'].apply(is_night)

In [15]:
data = (data.groupby('customer_id')
            .apply(lambda x: get_customer_spending_behaviour_features(
                x, windows_size_in_days=[1,7,30]
            )))

data = data.sort_values('tx_datetime').reset_index(drop=True)

In [16]:
data = (data.groupby('terminal_id')
            .apply(lambda x: get_count_risk_rolling_window(
                x, delay_period=7, windows_size_in_days=[1,7,30],
                feature='terminal_id'
            )))

data = data.sort_values('tx_datetime').reset_index(drop=True)

In [17]:
data.head()

,transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario,tx_during_weekend,...,customer_id_nb_tx_7day_window,customer_id_avg_amount_7day_window,customer_id_nb_tx_30day_window,customer_id_avg_amount_30day_window,terminal_id_nb_tx_1day_window,terminal_id_risk_1day_window,terminal_id_nb_tx_7day_window,terminal_id_risk_7day_window,terminal_id_nb_tx_30day_window,terminal_id_risk_30day_window
0,0,2022-05-02 00:00:31,1829,2355,22.35,31,0,0,0,0,...,1.0,22.35,1.0,22.35,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2022-05-02 00:01:59,157,7040,9.53,119,0,0,0,0,...,1.0,9.53,1.0,9.53,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2022-05-02 00:02:29,834,2170,82.26,149,0,0,0,0,...,1.0,82.26,1.0,82.26,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2022-05-02 00:04:47,2845,8301,25.29,287,0,0,0,0,...,1.0,25.29,1.0,25.29,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2022-05-02 00:07:04,1565,9269,31.69,424,0,0,0,0,...,1.0,31.69,1.0,31.69,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
DIR_OUTPUT = './data-transformed/'

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')

for day in range(data['tx_time_days'].max()+1):
    
    transactions_day = data[data['tx_time_days']==day].sort_values('tx_time_seconds')
    
    date = start_date + dt.timedelta(days=day)
    filename_output = date.strftime('%Y-%m-%d')+'.pkl'
    
    # Protocol=4 required for Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)

In [19]:
e_time = time.time() - t_script
e_time_format = str(dt.timedelta(seconds=np.round(e_time)))
print('Execution time: {}'.format(e_time_format))

Execution time: 0:19:31
